Run: docker compose up -d

In [33]:
import os

from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

In [ ]:
METRIC_TYPE = "angular" # euclidean, manhattan - Która metoda ma być użyta do porównywania
TREES_NUMBER = 10       # Ilość drzew do utworzenia
TEXT_TO_SEARCH = 30     # Numer sentencji do porównania
NN_NUMBER = 5           # Ilość sąsiadów do odszukania

In [35]:
# Zestaw danych - 10 przykładowych zdań
sentences = [
    "Słońce świeci na niebie.", # 0
    "Dzisiaj jest piękna pogoda.",
    "Lubię uczyć się programowania.",
    "Python to popularny język programowania.",
    "Wieczorem często czytam książki.",
    "Mój pies lubi biegać po parku.",
    "Programowanie to świetna zabawa.",
    "Niebo jest dziś bardzo niebieskie.",
    "Czytanie rozwija wyobraźnię.",
    "Chodzę codziennie na spacer z psem.",
    "Lubię Mazury, morze i góry.", # 10
    "Java to mój ulubiony język programowania.",
    "Ostatnio częściej programuję w Pythonie",
    "SQL też nie jest zły.",
    "Za tydzień jedziemy całą ekipą na wakacje w góry.",
    "Zawsze na wyjazdy wakacyjne zabieramy naszego psa.",
    "Dostałem wczoraj mandat za przektoczenie predkości.",
    "Jazda moim Porshe wyzwala we mnie mega emocje.",
    "Kolarstwo górskie daje mi wiele frajdy.",
    "Wczoraj byłem w kinie.",
    "Mam uczulenie na czekoladę ale za to mogę jeść marmoladę.", # 20
    "Najlepsze pączki są z marmoladą i lukrem.",
    "Twój tort urodzinowy był pyszny.",
    "Od roku ograniczam cukier i czuję sie wyśmienicie.",
    "Wczoraj kupiłem poradnik pt. Dieta dla każdego.",
    "Sposób odżywiania ma wpływ na nasze samopoczucie.",
    "Właśnie skończyłem czytać fajną książkę.",
    "Czytanie książek to moja ulubiona zabawa.",
    "Wczoraj byłem w bibliotek i wypożyczyłem kilka książek na temat programowania.",
    "Wczoraj wróciliśmy z wakacji z Mazur.",
    "Mój pies lubi gonić koty." # 30
]

In [36]:
# Ładowanie modelu do tworzenia osadzeń (embeddingów) zdań
model = SentenceTransformer('all-MiniLM-L6-v2')  # lekki i szybki model

# Tworzenie embeddingów dla każdego zdania
embeddings = model.encode(sentences)  # embedding = wektor liczb reprezentujących znaczenie zdania
vector_dim = len(embeddings[0])       # wymiar embeddingu, np. 384

# Ścieżka do pliku z indeksem
index_file = "index_annoy/sentences.ann"

# Tworzenie i budowanie indeksu tylko jeśli nie istnieje
if not os.path.exists(index_file):
    index = AnnoyIndex(vector_dim, METRIC_TYPE)
    for i, vec in enumerate(embeddings):
        index.add_item(i, vec)
    # Budowanie drzewa wyszukiwania (10 drzew – więcej = lepsza jakość, wolniejsze budowanie
    index.build(TREES_NUMBER)
    index.save(index_file)
    print(f"Indeks zapisany do pliku: {index_file}")
else:
    print(f"Indeks już istnieje: {index_file}")

# Tworzenie indeksu Annoy z metryką 'angular' (opartą na kącie między wektorami)
index = AnnoyIndex(vector_dim, METRIC_TYPE)
index.load(index_file)

Indeks już istnieje: index_annoy/sentences.ann


True

In [37]:
# Przykład zapytania – wyszukujemy zdania podobne do zdania nr 2
query_id = TEXT_TO_SEARCH
print(f"Zapytanie: {sentences[query_id]}")

Zapytanie: Mój pies lubi gonić koty.


In [38]:
# Wyszukiwanie 3 najbardziej podobnych zdań (bez samego zapytania)
nearest_ids = index.get_nns_by_item(query_id, NN_NUMBER, include_distances=True)

# Wyświetlenie wyników
print("\nNajbardziej podobne zdania:")
for idx, dist in zip(*nearest_ids):
    print(f"- \"{sentences[idx]}\" (odległość: {dist:.4f})")



Najbardziej podobne zdania:
- "Mój pies lubi gonić koty." (odległość: 0.0000)
- "Mój pies lubi biegać po parku." (odległość: 0.6589)
- "Dzisiaj jest piękna pogoda." (odległość: 0.9895)
- "Czytanie książek to moja ulubiona zabawa." (odległość: 1.0453)
- "Kolarstwo górskie daje mi wiele frajdy." (odległość: 1.0627)
